In [2]:
import os
import subprocess
import threading
import tempfile
import uuid
import time
import whisper
import customtkinter as ctk
import tkinter as tk
from tkinter import filedialog, ttk
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import OllamaLLM
from langchain_groq import ChatGroq
from PIL import Image, ImageTk
from customtkinter import CTkImage
from groq import Groq

In [3]:
# ------------------ Utility Functions ------------------
def _parse_srt_time(t):
    try:
        h, m, s_ms = t.split(":")
        if "," in s_ms:
            s, ms = s_ms.split(",")
            return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000
        else:
            return int(h) * 3600 + int(m) * 60 + float(s_ms)
    except:
        pass
    try:
        m, s = t.split(":")
        return int(m) * 60 + float(s)
    except:
        pass
    try:
        return float(t)
    except:
        raise ValueError(f"Unrecognized timestamp format: {t}")

def format_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = seconds % 60
    return f"{h:02}:{m:02}:{s:06.3f}".replace(".", ",")

# ------------------ FFmpeg Functions ------------------
def concatenate_videos(video_paths, output_filepath):
    try:
        if not video_paths:
            return "❌ Error: No video clips provided.", None
        ffmpeg_path = "C:\\ffmpeg\\ffmpeg.exe"
        temp_dir = tempfile.gettempdir()
        list_file_path = os.path.join(temp_dir, f"concat_list_{uuid.uuid4().hex[:8]}.txt")
        with open(list_file_path, "w", encoding="utf-8") as f:
            for path in video_paths:
                f.write(f"file '{path.replace('\\', '/')}\n")
        command = [
            ffmpeg_path, "-y", "-f", "concat", "-safe", "0",
            "-i", list_file_path, "-c", "copy", output_filepath
        ]
        result = subprocess.run(command, capture_output=True, text=True)
        os.remove(list_file_path)
        if result.returncode != 0:
            return f"❌ FFmpeg error:\n{result.stderr}", None
        return f"✅ Concatenated video saved to: {output_filepath}", output_filepath
    except Exception as e:
        return f"❌ Exception during concatenation: {str(e)}", None

# ------------------ LLM Setup ------------------
os.environ["GROQ_API_KEY"] = "Your_API_Key"

llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7)

general_template = """
You are Vivi, an expert and friendly video assistant chatbot.
You are having an ongoing conversation with the user. You have access to a full transcript of a video only when it is uploaded. If the user’s question is about the video, answer helpfully and refer to timestamps if relevant.
If the question is general and not related to the video, just respond helpfully like a normal assistant.
---
Conversation History:
{context}
---
Full Transcript of the Video:
{transcript}
---
User:
{question}
---
Vivi:
"""

clipping_template = """
You are an expert video analysis assistant. Given a user query and the transcript of a video with timestamps, identify all timestamp ranges where the video content is relevant to the query. Return the result as a list of timestamp ranges (start and end times in seconds) and a brief explanation of why each range is relevant.

Return timestamps as plain numbers with two decimal places (e.g., 31.00, 45.00) without brackets or other characters. Ensure the format is consistent. Ensure end_time is greater than start_time and both are non-negative.

Query: {query}
Transcript: {transcript}

Return the result in the following format and each range and its explanation must be on separate lines:
- Range: start_time - end_time
- Range: start_time - end_time
  Relevance: [Brief explanation of why this range is relevant to the query]
"""

prompt_general = ChatPromptTemplate.from_template(general_template)
prompt_clipping = ChatPromptTemplate.from_template(clipping_template)
chain_general = prompt_general | llm
chain_clipping = prompt_clipping | llm

# ------------------ Vivi GUI Class ------------------
class ViviChatbot:
    def __init__(self):
        # Use light appearance mode
        ctk.set_appearance_mode("light")
        ctk.set_default_color_theme("blue")

        # Create root and set background
        self.root = ctk.CTk()
        self.root.configure(fg_color="#FFFFFF")
        self.root.title("ClipQuery- Video Chatbot")
        self.root.geometry("900x650")

        self.video_path = ""
        self.context = ""
        self.transcript_segments = []
        self.full_transcript_text = ""
        self.audio_process = None

        logo_frame = ctk.CTkFrame(self.root, fg_color="#FFFFFF")
        logo_frame.pack(padx=20, pady=(10, 0), anchor="w")

        self.logo_image = CTkImage(Image.open("vivi_logo_1.png"), size=(40, 40))
        logo_label = ctk.CTkLabel(logo_frame, image=self.logo_image, text="")
        logo_label.image = self.logo_image  # 🔒 Prevent TclError
        logo_label.pack(side="left", padx=(0, 10))

        logo_text = ctk.CTkLabel(
            logo_frame,
            text="ClipQuery",
            font=("Arial", 28, "bold"),
            text_color="#1a2238",
            anchor="w",  # Make sure it's left-anchored
            justify="left"
        )
        logo_text.pack(side="left")
        
        # Chat display frame
        self.chat_frame = ctk.CTkScrollableFrame(self.root, width=880, height=500, fg_color="#FFFFFF")
        self.chat_frame.pack(padx=20, pady=(5, 10), fill="both", expand=True)
        self.scrollable_frame = self.chat_frame

        # Entry field frame
        self.entry_frame = ctk.CTkFrame(self.root, fg_color="#FFFFFF")
        self.entry_frame.pack(pady=(0, 5), fill="x", padx=20)

        self.user_entry = ctk.CTkEntry(self.entry_frame, placeholder_text="Ask Vivi...", width=700, height=40, font=("Arial", 16))
        self.user_entry.pack(side="left", fill="x", expand=True, padx=(0, 10))
        self.user_entry.bind("<Return>", lambda e: self.send_message())

        self.send_btn = ctk.CTkButton(self.entry_frame, text="Send", command=self.send_message, fg_color="#1a2238", width=160, height=40, font=("Arial", 16))
        self.send_btn.pack(side="right")

        # Buttons for upload, input video, and final video
        self.btn_frame = ctk.CTkFrame(self.root, fg_color="#FFFFFF")
        self.btn_frame.pack(pady=(0, 10))

        self.upload_btn = ctk.CTkButton(self.btn_frame, text="📂 Upload Video", command=self.browse_video, fg_color="#1a2238", width=160, height=30, font=("Arial", 16))
        self.upload_btn.pack(side="left", padx=5)

        self.input_video_btn = ctk.CTkButton(self.btn_frame, text="▶️Input Video", command=self.play_input_video, fg_color="#1a2238", width=160, height=30, font=("Arial", 16))
        self.input_video_btn.pack(side="left", padx=5)

        self.final_video_btn = ctk.CTkButton(self.btn_frame, text="🎬 Final Video", command=self.play_final_video, fg_color="#1a2238", width=160, height=30, font=("Arial", 16))
        self.final_video_btn.pack(side="left", padx=5)
        
        # Initial welcome message
        self.display_message("system", "🎬 Welcome to Vivi!\nType 'exit' to quit.\nUse format `Clip:<your query>` for semantic editing.")

    def typewriter_effect(self, sender, message):
        # Outer frame to control side alignment
        outer_frame = ctk.CTkFrame(self.scrollable_frame, fg_color="#FFFFFF")
        outer_frame.pack(fill="x", padx=10, pady=5)

        # Inner chat bubble with side-specific color
        bubble_frame = ctk.CTkFrame(
            outer_frame,
            fg_color="#3a0e2e" if sender.lower() == "vivi" else "#1B263B",
            corner_radius=12
        )
        if sender.lower() == "vivi":
            bubble_frame.pack(anchor="w", padx=5)
        else:
            bubble_frame.pack(anchor="e", padx=5)

        # Name label (optional, can be hidden if not needed)
        name_label = ctk.CTkLabel(
            bubble_frame,
            text=sender + ":",
            font=("Arial", 12, "bold"),
            text_color="#F0F0F0"
        )
        name_label.pack(anchor="w", padx=8, pady=(6, 0))

        # Message label for typewriter text
        message_label = ctk.CTkLabel(
            bubble_frame,
            text="",
            font=("Arial", 20, "normal"),
            wraplength=500,
            justify="left",
            text_color="#F0F0F0"
        )
        message_label.pack(anchor="w", padx=8, pady=(0, 8))

        # Scroll to bottom before typing
        self.root.update_idletasks()
        self.chat_frame._parent_canvas.yview_moveto(1.0)

        # Typewriter animation
        current_text = ""
        for word in message.split():
            current_text += word + " "
            message_label.configure(text=current_text)
            self.root.update_idletasks()
            self.chat_frame._parent_canvas.yview_moveto(1.0)
            time.sleep(0.04)
    
    def display_message(self, sender, message):
        outer_frame = ctk.CTkFrame(self.chat_frame, fg_color="#FFFFFF")
        outer_frame.pack(fill="x", padx=10, pady=5)

        bubble = ctk.CTkFrame(
            outer_frame,
            corner_radius=15,
            fg_color="#3a0e2e" if sender.lower() != "user" else "#1a2238"
        )

        if sender.lower() == "user":
            bubble.pack(anchor="e", padx=5)
        else:
            bubble.pack(anchor="w", padx=5)

        label = ctk.CTkLabel(
            bubble,
            text=f"{message}",
            wraplength=700,
            font=("Arial", 20, "normal"),
            justify="left",
            text_color="#F0F0F0"
        )
        label.pack(anchor="w", padx=10, pady=5)


    def clip_video(self, start_time, end_time):
        try:
            if start_time >= end_time:
                self.display_message("system", f"⚠ Invalid clip range: {start_time} >= {end_time}")
                return "Invalid time range", None
            clip_filename = f"clip_{uuid.uuid4().hex[:8]}.mp4"
            output_path = os.path.join(tempfile.gettempdir(), clip_filename)
            subprocess.run([
                "ffmpeg", "-y", "-ss", str(start_time), "-to", str(end_time),
                "-i", self.video_path, "-c", "copy", output_path
            ], check=True)
            return f"🎬 Video clip saved to {output_path}", output_path
        except Exception as e:
            self.display_message("system", f"❌ Error clipping video: {e}")

    def transcribe_video(self, video_path):
            base = os.path.splitext(os.path.basename(video_path))[0]
            dir_ = os.path.dirname(video_path)
            txt_path = os.path.join(dir_, f"{base}.txt")
            srt_path = os.path.join(dir_, f"{base}.srt")

            # Check if already transcribed
            if os.path.exists(txt_path) and os.path.exists(srt_path):
                with open(srt_path, "r", encoding="utf-8") as f:
                    blocks = f.read().strip().split("\n\n")
                with open(txt_path, "r", encoding="utf-8") as f:
                    timed_transcript = f.read().strip()
                segments = []
                for block in blocks:
                    lines = block.split("\n")
                    if len(lines) >= 3:
                        times = lines[1].split(" --> ")
                        start = _parse_srt_time(times[0])
                        end = _parse_srt_time(times[1])
                        text = " ".join(lines[2:])
                        segments.append({"start": start, "end": end, "text": text})
                return timed_transcript, segments

            # Use Groq Whisper API
            self.display_message("system", "🔍 Running Whisper (large-v3) for transcription...")

            api_key = os.environ.get("GROQ_API_KEY")
            if not api_key:
                raise ValueError("GROQ_API_KEY is not set in environment variables.")

            client = Groq(api_key=api_key)

            with open(video_path, "rb") as file:
                transcription = client.audio.transcriptions.create(
                    file=file,
                    model="whisper-large-v3",
                    prompt="",  # You can provide custom prompt if needed
                    response_format="verbose_json",
                    timestamp_granularities=["segment"],
                    language="en",
                    temperature=0.0
                )

            segments = []
            timed_transcript = ""
            for seg in transcription.segments:
                start = seg["start"]
                end = seg["end"]
                text = seg["text"]
                segments.append({"start": start, "end": end, "text": text})
                timed_transcript += f"[{start:.2f} - {end:.2f}] {text}\n"

            # Save to SRT
            with open(srt_path, "w", encoding="utf-8") as f:
                for i, seg in enumerate(segments):
                    f.write(f"{i+1}\n{format_time(seg['start'])} --> {format_time(seg['end'])}\n{seg['text']}\n\n")

            # Save to TXT
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(timed_transcript)

            return timed_transcript, segments

    def send_message(self):
        user_input = self.user_entry.get()
        if user_input.strip().lower() == "exit":
            self.root.destroy()
            return
        self.display_message("user", user_input)
        self.user_entry.delete(0, tk.END)
        self.user_entry.configure(state="disabled")
        self.send_btn.configure(state="disabled")

        def extract_content(response):
            return str(response.content) if hasattr(response, "content") else str(response)

        def run_bot():
            if user_input.lower().startswith("clip:"):
                query = user_input[len("clip:"):].strip()
                response = chain_clipping.invoke({"query": query, "transcript": self.transcript_segments})
                response_text = extract_content(response)

                video_clips = []
                for line in response_text.strip().split("\n"):
                    if line.startswith("- Range: "):
                        parts = line[len("- Range: "):].split(" - ")
                        try:
                            start = float(parts[0])
                            end = float(parts[1])
                            msg, clip_path = self.clip_video(start, end)
                            if clip_path:
                                video_clips.append(clip_path)
                        except:
                            continue
                if video_clips:
                    msg, out = concatenate_videos(
                        video_clips,
                        output_filepath=os.path.join(os.path.dirname(self.video_path), "final_output.mp4")
                    )
                    self.display_message("Vivi", msg)
            else:
                response = chain_general.invoke({
                    "context": self.context,
                    "question": user_input,
                    "transcript": self.transcript_segments
                })
                response_text = extract_content(response)
                self.typewriter_effect("Vivi", response_text)
                self.context += f"\nUser: {user_input}\nAI: {response_text}\n"

            self.user_entry.configure(state="normal")
            self.send_btn.configure(state="normal")
            self.user_entry.focus()

        threading.Thread(target=run_bot).start()

    def browse_video(self):
        self.video_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4 *.mov *.avi")])
        if not self.video_path:
            return
        self.display_message("system", f"📁 Selected video: {os.path.basename(self.video_path)}")
        def process_video():
            self.full_transcript_text, self.transcript_segments = self.transcribe_video(self.video_path)
            self.display_message("system", "✅ Transcription completed!")
        threading.Thread(target=process_video).start()

    def play_input_video(self):
        if not self.video_path:
            self.display_message("system", "⚠️ No video uploaded yet. Upload a video first.")
            return
        if self.audio_process and self.audio_process.poll() is None:
            self.audio_process.terminate()
        try:
            self.audio_process = subprocess.Popen(["ffplay", "-autoexit", "-loglevel", "quiet", self.video_path])
        except Exception as e:
            self.display_message("system", f"❌ Failed to play input video: {e}")

    def play_final_video(self):
        final_output = os.path.join(os.path.dirname(self.video_path), "final_output.mp4")
        if not os.path.exists(final_output):
            self.display_message("system", "⚠️ No final video available. Generate a clipped video first.")
            return
        if self.audio_process and self.audio_process.poll() is None:
            self.audio_process.terminate()
        try:
            self.audio_process = subprocess.Popen(["ffplay", "-autoexit", "-loglevel", "quiet", final_output])
        except Exception as e:
            self.display_message("system", f"❌ Failed to play final video: {e}")

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = ViviChatbot()
    app.run()


SyntaxError: f-string expression part cannot include a backslash (283306850.py, line 38)